# 1. Preparation

## 1.1 Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp -r /content/drive/MyDrive/Uni\ Leipzig/SS2022/.ssh ~/

## 1.2 Get correct CuDNN version
See https://github.com/googlecolab/colabtools/issues/2600



In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

## 1.3 Install necessary packages

In [ ]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [ ]:
!python3 -m pip install -r '../requirements.txt'

In [41]:
import re

def para(model, tokenizer, row, anchor_col="sentence1", para_col="sentence2", label_col="label", path="glue", name="mrpc", split="train"):
  out_row = {
             "anchor": row[anchor_col],
             "para": row[para_col],
             "neg_1": "",
             "original_label": row[label_col],
             "contains_numeric": False,
             "path": path,
             "name": name,
             "split": split}

  # return samples which contain paraphrase as is
  if row[label_col] == 1:
        return out_row


  # move hard negative to neg_1
  out_row["neg_1"] = row[para_col]
  #mark rows with digits in anchor sentence, leads to logical errors in paraphrase
  if re.search("\d", row[anchor_col]):
        out_row["contains_numeric"] = True


  anchor = row[anchor_col]
  anchor = re.sub(r'\s([\.,!\?\'"])', r'\1', anchor)
  anchor = anchor.replace('" ', '"')


  encoding = tokenizer.encode("paraphrase: " + anchor, return_tensors="pt")
  input_ids = encoding.to(device)


  paraphrases = model.generate(
      input_ids=input_ids,
      max_length=128,
      early_stopping=True,
      num_beams=3,
      num_return_sequences=1

  )
  	
  para = tokenizer.decode(paraphrases[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

  # remove first tokens from anchor and paraphrase, that contain "paraphrsedoutput: " and "paraphrase: "
  para_token_list =paraphrases[0].tolist()[7:]
  input_token_list = input_ids.tolist()[0][3:]

  # check if generated tokens are the same as in original sentence, which means sentence was not altered or just shuffeled.
  # Sentences like this are not used
  # remove paraphrases without new tokens and empty ones
  if not para or sorted(para_token_list) ==sorted(input_token_list) or len(para_token_list) == 128:
    out_row["para"] = "None"
    return out_row
  else:
      para = para.replace("paraphrasedoutput: ", "")
      out_row["para"] = para
    
  return out_row

In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
model.to(device)
print(" ")


In [ ]:
import pandas as pd
dataset_definitions = [
    #{"path": "glue", "name": "mrpc"},
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}
]
out = {"train":[], "validation":[], "test":[]}
for definition in dataset_definitions:
  dataset = datasets.load_dataset(path=definition["path"], name=definition["name"])
  for split in ["train", "validation", "test"]:
    if split not in dataset:
      continue
    dataset_part = pd.DataFrame(dataset[split])
    #dataset_post = dataset_part.map(lambda x: para(model, tokenizer, x), num_proc=1)
    for idx, row in dataset_part.iterrows():
      out[split].append(para(model, tokenizer,row, split=split, path=definition["path"], name=definition["name"]))
for key, value in out.items():
    df = pd.DataFrame(value)
    df = df[df["para"] != "None"]
    df = df.reset_index()
    df.to_csv(f"out_{key}_paws.csv", index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
for split in ["train", "validation", "test"]:
    anchors_with_para = pd.read_csv(f"out_{split}_paws.csv", index_col="index").fillna("")

    df = {"docno": [], "text": []}
    for index, row in anchors_with_para.iterrows():
        for sent in ["anchor", "para", "neg_1"]:
            if sent and sent.strip() !="" and sent != "None":
                df["text"].append(row[sent])
                df["docno"].append(str(index) + "_" + sent)
    df = pd.DataFrame.from_dict(df)
    print(df)
    # index the text, record the docnos as metadata
    pd_indexer = pt.DFIndexer(f"./pd_index_{split}")
    indexref = pd_indexer.index(df["text"], df["docno"])

    batch_ret = pt.BatchRetrieve(indexref,num_results=10)
    out_df = {"anchor":[], "para":[], "neg1":[], "neg2":[], "neg3":[], "neg4":[], "neg5":[], "neg6":[]}
    for idx_row,row in anchors_with_para.iterrows():
        out_df["anchor"].append(row["anchor"])
        out_df["para"].append(row["para"])
        
        if row["neg_1"] != "":
            neg_count = 2
            out_df["neg1"].append(row["neg_1"])
        else:
            neg_count=1

        s = re.sub(r'[^a-zA-Z0-9\s]', ' ', row["anchor"])

        res = batch_ret(s)
        if res.shape[0] < 10:
            print(res)
        for res_num ,r in res.iterrows():
            idx_num, sent = r["docno"].split("_", 1)[0],  r["docno"].split("_", 1)[1]
            if int(idx_num) != idx_row:
                out_df[f"neg{neg_count}"].append(anchors_with_para.loc[int(idx_num)][sent])
                neg_count += 1
            if neg_count == 7:
                break
        if neg_count < 7:
            for n in range(neg_count, 7):
                out_df[f"neg{n}"].append("")

    out_df = pd.DataFrame(out_df)
    out_df.to_csv(f"out_{split}.csv")

In [ ]:
import datasets
import pandas as pd
import csv
import numpy as np
from random import shuffle
from parrot import Parrot
import torch
from nltk.util import ngrams
import nltk
from nltk.corpus import stopwords

# =============================================================
# Hard negative mining
# =============================================================
class HardNegativePreparer():
    """
    This class is used by the contrastive learning_manager to get negative samples for specified sentences.
    It takes the sentences from the csv "Negative_Sentences.csv" created by the HardNegativeFinder
    """
    def __init__(self, csv_path="Negative_Sentences_Sig300.csv"):
        self.df = pd.read_csv(csv_path)

    def build_dataset_with_negatives(self, dataset: datasets.Dataset, n=1):
        """
        :param dataset: Dataset with two sentences and a label
        :param n:       Number of negatives to add
        :return:        Dataset that only contains the pairs that have a label of 1.
                        Additionally, n more sentences were added
        """
        # Filter only positive samples
        filtered_ds = dataset.filter(lambda x: x["label"] == 1)

        # Add the hard negatives to each ds separately
        train_ds = filtered_ds["train"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="train"))
        val_ds = filtered_ds["validation"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="val"))
        test_ds = filtered_ds["test"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="test"))

        # Combine the three datasets into one DatasetDict and return it
        return datasets.DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

    def get_hard_negatives(self, sen_id: int, n=1, dataset="train"):
        """
        Takes in a sentence i and returns n hard negatives as obtained by the HardNegativeFinder
        :param sen_id:  The id for which to return hard negatives
        :param n:       The number of hard negatives to return
        :param dataset: The name of the dataset the sentence belongs to
        :return:        A dictionary of hard negative sentences (starting with sentence3, sentence4, ...)
        """
     
        # Define the columns of self.df based on n
        columns = [str("sen_" + str(idx)) for idx in range(n)]

        # Get the correct row by matching the sen_id on anchor_idx and the dataset name
        row = self.df.loc[((self.df["anchor_idx"] == "sen1_" + str(sen_id)) & (self.df["dataset"]==dataset))]

        # Get the sentences as a list and turn them into a dictionary starting with key sentence3
        sentences = row[columns].values.tolist()[0]
        result_dict = dict(zip(["sentence" + str(idx + 3) for idx in range(n)], sentences))
        return result_dict
    

    def build_dataset_with_positives_and_negatives(self, dataset: datasets.Dataset, n=1):
        """
        :param dataset: Dataset with two sentences and a label
        :param n:       Number of negatives to add
        :return:        Augmented Dataset. Negative labeled Sentences get one positive paraphrase added and are filled to reach n negatives
                        For already positive pairs, only negatives are created.
        """
        # Filter only negative samples, for which positive is created
        #filtered_ds = dataset.filter(lambda x: x["label"] == 0)

        augmented_with_paraphrases = generate_paraphrases(dataset)
        augmented_with_paraphrases.to_csv("dataset_with_para.csv")
        return


        # Add the hard negatives to each ds separately
        train_ds = augmented_with_paraphrases["train"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="train"))
        val_ds = augmented_with_paraphrases["validation"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="val"))
        test_ds = augmented_with_paraphrases["test"].map(lambda x: self.get_hard_negatives(sen_id=x["idx"], n=n, dataset="test"))

        # Combine the three datasets into one DatasetDict and return it
        return datasets.DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})



def paraphrase(parrot, row):
    '''
    :param parrot: Instance of Parrot paraphrase generator
    :param row: dataset row with one anchor sentence and one negative paraphrase sample
    :return:    dataset row with the anchor, one positive sample and the negative sample moved to column "sen_2"
    '''
    print(row["index"])
    if row["label"] == 1:
        row["sen_2"] = ""
        return row
    row["sen_2"] = row["sentence2"]
    row["sen_1"] = parrot.augment(row["sentence1"])
    return row

def generate_paraphrases(dataset: datasets.Dataset):
    
    #uncomment to get reproducable paraphrase generations
    def random_state(seed):
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

        random_state(1234)
    

    parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)
    out = []
    for row in dataset:
        out_row= paraphrase(parrot, row)
        out.append(out_row)
    #dataset = dataset.map(lambda x: paraphrase(parrot, x), num_proc=1)

    return dataset






def construct_q_gram_set(string: str, q=3, q_padding=False):
    s = set()

    # Apply padding if specified
    if q_padding:
        string = "#"*(q-1) + string + "#"*(q-1)

    string = string.lower()

    # If the string is shorter than Q, apply padding at the end
    if len(string) < q:
        string= string + str("#"*(q-len(string)))

    for i,_ in enumerate(string):
        q_gram = string[i:min(i+q, len(string))]
        if len(q_gram)==q:
            s.add(q_gram)

    return s

def construct_n_gram_set(s: str, all_n=[1, 2], filter_stopwords=False):
    out = set()
    s = s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)

    tokens = [token for token in s.split(" ") if token != ""]
    if filter_stopwords:
        tokens = [token for token in tokens if token not in stopwords.words('english')]
    for n in all_n:
         
        out |= set(ngrams(tokens, n))
    return out


class HardNegativeFinder():
    """
    This class is used to find hard negative sentences for sentences in the provided dataset.
    These are written to Negative_Sentences.csv.
    """
    def __init__(self, ds):
        self.train_ds = ds[ds["split"] == "train"]
        self.val_ds = ds[ds["split"] == "validation"]
        self.test_ds = ds[ds["split"] == "test"]

    def create_qgrams(self):
        """
        For all positive pairs, only sentence 1 is turned into a Q-gram, because this is the anchor sentence.
        Then, both sentences of all negative pairs are also turned into Q-grams as matching candidates for the anchors.

        In addition, all q-grams are saved to build the vocabulary used by LSH
        """
        result_dict = {}
        loop_list = [{"name": "train", "ds": self.train_ds}, {"name": "val", "ds": self.val_ds},
                {"name": "test", "ds": self.test_ds}]
        print("Creating QGrams")
        for pair in loop_list:
            vocab = set()
            anchor_sen = []
            anchor_idx = []
            candidate_sen = []
            candidate_idx = []
            for idx, row in pair["ds"].iterrows():

                # Get the Q-gram sets for all anchor sentences
                sen1 = row["anchor"]
                #q_gram_set = construct_q_gram_set(sen1)
                q_gram_set =  construct_n_gram_set(sen1)
                #print(q_gram_set)
                # Add the set to the anchor and update the vocabulary
                anchor_sen.append(q_gram_set)
                vocab.update(q_gram_set)

                # Add the idx to identify sentences
                anchor_idx.append("anchor_" + str(idx))

                candidate_sen.append(q_gram_set)
                candidate_idx.append("anchor_" + str(idx))

                # Depending on original label, add original Paraphrase or neg_1 to vocab
                sen2_label = "para" if row["original_label"] == 1 else "neg_1"
                sen2 = row[sen2_label]

                #q_gram_set2 = construct_q_gram_set(sen2)
                q_gram_set2 =  construct_n_gram_set(sen2)

                # Add the sets to the candidates and the vocabulary
                candidate_sen.append(q_gram_set2)
                vocab.update(q_gram_set2)

                # Add the idx to identify sentences
                candidate_idx.append(sen2_label + "_" + str(idx))


            # Create the Hasher for the current dataset based on the vocabulary and save the Q-grams
            Hasher = Locality_Sensitive_Hasher(vocab=vocab, num_signatures=200)

            # Add both lists and the hasher to the result_dict
            result_dict[pair["name"]] = {"anchors": {"idx": anchor_idx,
                                                    "sen": anchor_sen},
                                        "candidates": {"idx": candidate_idx,
                                                        "sen": candidate_sen},
                                        "hasher": Hasher}

            self.q_gram_dict = result_dict


    def find_negatives(self):
        if not hasattr(self, "q_gram_dict"):
            self.create_qgrams()

        # Get the signatures for the anchors and candidates for all three datasets
        self.get_signatures()

        # Initialize a dictionary
        negatives_dict = {}

        for ds_name, ds_dict in self.q_gram_dict.items():

            print("\n\n" + "=" * 50)
            print(f"Forming matches for {ds_name}")
            print("=" * 50)

            anchor_sig = ds_dict["anchors"]["sig"]
            candidate_sig = ds_dict["candidates"]["sig"]
            anchor_idx = ds_dict["anchors"]["idx"]
            candidate_idx = ds_dict["candidates"]["idx"]

            # Initialize a sub dictionary for the current ds_name
            ds_match_dict = {}

            # For each of the anchor signatures, find matches in the candidate signatures
            num_anchors = len(anchor_sig)
            for i, sig_vector in enumerate(anchor_sig):
                print(f"- Anchor {i + 1}/{num_anchors}")

                # If an element in a row of candidate sig matches the element in sig_vector, its value is set to true
                # sum(axis=1) counts the number of True values per row
                match_vector = (candidate_sig==sig_vector).sum(axis=1)

                # Combine the number of matches in match_vector with the candidate idx into a dictionary
                # Save the dict in the ds_match_dict (key: corresponding anchor idx)
                ds_match_dict[anchor_idx[i]] = dict(zip(candidate_idx, list(match_vector)))
            negatives_dict[ds_name] = ds_match_dict

        # Set the negatives_dict attribute
        self.negatives_dict = negatives_dict


    def get_signatures(self):
        """
        Uses the Hasher-instance in each sub-dictionary of self.q_gram_dict to create signature matrices for
        anchors and candidates.
        One row of the matrix corresponds to one sentence
        """
        for ds_name, ds_dict in self.q_gram_dict.items():

            print("\n\n" + "="*50)
            print(f"Getting signatures for {ds_name}")
            print("=" * 50)

            # Get the signature matrix for both the anchors and the candidates
            Hasher = ds_dict["hasher"]

            print("Anchor sentences:")
            anchor_sig = Hasher.create_signature_matrix(ds_dict["anchors"]["sen"])

            print("\nCandidate sentences:")
            candidate_sig = Hasher.create_signature_matrix(ds_dict["candidates"]["sen"])

            # Add signatures to the dictionary
            ds_dict["anchors"]["sig"] = anchor_sig
            ds_dict["candidates"]["sig"] = candidate_sig


    def write_negatives(self, n=50, out_path="Negative_Sentences.csv"):
        """
        Function that uses the negatives identified in find_negatives and writes the sentences out to a csv-File
        Process per anchor sentence
        1. Sort the dictionary by value (number of matching signatures)
        2. Take the n sentences with the highest number of matches
        3. Identify the sentences in the dataset by their index and sentence number
        4. Write a new row into the CSV ([ds_type], [anchor_idx], [sentence1], [sentence2], ..., [sentence_N])

        :param n:           Number of sentences to be stored per anchor
        :param out_path:    Path to the csv that stores the matches
        """

        if not hasattr(self, "negatives_dict"):
            self.find_negatives()

        with open(out_path, "w") as file:
            sentence_columns = [("sen_" + str(num)) for num in range(n)]
            header = "dataset,anchor_idx," + ",".join(sentence_columns) + "\n"
            file.write(header)


        for ds_name, ds_dict in self.negatives_dict.items():
            for anchor_idx, num_match_dict in ds_dict.items():
                # 1. Get the candidate_idx as a list sorted by number of matches
                sorted_idx = list(dict(sorted(num_match_dict.items(), key=lambda x:x[1], reverse=True)).keys())

                # 2. and 3. Get the first n sentences for the sorted_idx list
                top_n_sentences = self.identify_sentences(sorted_idx=sorted_idx, n=n, ds_name=ds_name) 

                # 4. Define the csv row and append it to the file
                csv_row = [ds_name, anchor_idx, *top_n_sentences]

                with open(out_path, "a") as file:
                    writer = csv.writer(file)
                    writer.writerow(item for item in csv_row)


    def identify_sentences(self, sorted_idx: list, n, ds_name):
        """
        Takes in a list of indices and a number of sentences.
        Identifies the sentences in the dataset and returns them as a list
        :param sorted_idx:      List of candidate indices; Sorted by number of matches with an anchor sentence
        :param n:               How many sentences to return
        :param ds_name:         Name of the dataset in which to look for the sentences
        :return:                List of the sentences with the highest number of matches
        """

        # Keep a maximum of n indices
        num_sentences = min(len(sorted_idx), n)
        sorted_idx = sorted_idx[:num_sentences]

        # Turn the sentence indices into the correct format for the dataset
        # - The number at the end is the idx in the ds
        # - sen1 or sen2 identifies the sentence in the pair
        index_nums = [int(full_idx.split("_")[-1]) for full_idx in sorted_idx]
        sen_keys = ["para" if full_idx.split("_")[0] == "para" else "neg_1" for full_idx in sorted_idx]

        # Filter the correct dataset to only contain sentences in the sorted_idx
        ds = getattr(self, ds_name + "_ds")

        # Get the sentences based on their position in the ds and whether it is sentence1 or sentence2
        result_list = []
        for i, position in enumerate(index_nums):
            #sentence = ds[position][sen_keys[i]]
            sentence = ds.loc[position][sen_keys[i]]
            result_list.append(sentence)

        return result_list


def create_vocab_dict(vocab):
    # Create a dictionary to map vocab elements to indices
    vocab_dict = {}
    for i, q_gram in enumerate(vocab):
        vocab_dict[q_gram] = i

    return vocab_dict

# ==============================================================
# LSH to speed up the comparison
# ==============================================================
class Locality_Sensitive_Hasher():
    def __init__(self, vocab, num_signatures=100):
        """
        :param vocab:           Set of all q-grams in the sets to be encoded
        :param num_signatures:  How many signatures should be used to represent one q-gram set
        """

        self.vocab = vocab
        print('Setting vocabulary...')
        self.vocab_dict = create_vocab_dict(self.vocab)
        self.num_signatures = num_signatures

        print('Creating hash functions...')
        self.update_lsh_hash_funcs()

    def update_vocab(self, vocab):
        self.vocab = vocab
        self.vocab_dict = create_vocab_dict(self.vocab)
        print('Vocabulary updated')

        print('Updating hash functions with new vocabulary...')
        self.update_lsh_hash_funcs()


    def update_lsh_hash_funcs(self):
        self.hash_list = []
        for i in range(self.num_signatures):
            # Create a randomized list of numbers from one to the length of the vocabulary
            hash_func = list(range(1, len(self.vocab) + 1))
            shuffle(hash_func)
            self.hash_list.append(hash_func)


    def create_signature_matrix(self, q_grams: list):
        """
        Takes in a list of Q-grams and returns a matrix of signatures
        :param q_grams:     List of Q-grams
        :return:            Numpy array of signatures
        """

        num_elem = len(q_grams)
        sig_matrix = np.zeros((num_elem, self.num_signatures))

        for i, q_gram in enumerate(q_grams):
            if i%20 == 0:
                print(f"- Sentence {i+1}/{num_elem}")
            sparse_vector = self.create_sparse_vector(q_gram)
            signature = self.create_dense_vector(sparse_vector)

            sig_matrix[i:i+1, :] = signature

        return sig_matrix


    def create_sparse_vector(self, q_gram_set):
        '''
        Takes in a string
        :param q_gram_set:      Set of q-grams to be turned into a sparse one-vector using self.vocab_dict
        :return:                One-hot vector corresponding to the provided value as numpy array
        '''

        val_one_hot = np.zeros(len(self.vocab))
        # Identify the index of each q_gram in the dictionary and set the corresponding element to 1
        for q_gram in q_gram_set:
            ind = self.vocab_dict[q_gram]
            val_one_hot[ind:(ind + 1)] = 1

        return val_one_hot


    def create_dense_vector(self, one_hot_vector):
        '''
        Creates a sparse signature vector using self.hash_list
        :param one_hot_vector:      Sparse one-hot vector to be turned into a dense signature vector
        :return:                    Dense signature vector as numpy array
        '''
        sig_vec = np.zeros(self.num_signatures, dtype=int)
        for sig_post, func in enumerate(self.hash_list):
            for i in range(1, len(self.vocab)+1):
                # Obtain the index of i in the hash-function (first iteration looks for the position of 1, then 2, ...)
                idx = func.index(i)
                vec_value = one_hot_vector[idx]

                # If the value at that position in the vector is 1, append the signature value
                # and proceed with the next hash
                if vec_value==1:
                    sig_vec[sig_post:(sig_post+1)] = i
                    break

        return sig_vec

In [ ]:
# from HardNegativeFinder import HardNegativeFinder
anchors_with_para = pd.read_csv("out_paws_train.csv", index_col="index", nrows=200)
nltk.download('stopwords')
finder = HardNegativeFinder(anchors_with_para)

finder.find_negatives()

finder.write_negatives(n=50, out_path="Negative_Sentences_paws_with_anchors.csv")

In [ ]:
!pip install python-terrier

In [1]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset


In [2]:
import pyterrier as pt
import pandas as pd
if not pt.started():
  pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [4]:
for split in ["train", "validation", "test"]:
    anchors_with_para = pd.read_csv(f"out_[split}_paws.csv", index_col="index").fillna("")

    df = {"docno": [], "text": []}
    for index, row in anchors_with_para.iterrows():
        for sent in ["anchor", "para", "neg_1"]:
            if sent and sent.strip() !="" and sent != "None":
                df["text"].append(row[sent])
                df["docno"].append(str(index) + "_" + sent)
    df = pd.DataFrame.from_dict(df)
    print(df)
    # index the text, record the docnos as metadata
    pd_indexer = pt.DFIndexer(f"./pd_index_{split}")
    indexref = pd_indexer.index(df["text"], df["docno"])

    batch_ret = pt.BatchRetrieve(indexref,num_results=10)
    out_df = {"anchor":[], "para":[], "neg1":[], "neg2":[], "neg3":[], "neg4":[], "neg5":[], "neg6":[]}
    for idx_row,row in anchors_with_para.iterrows():
        out_df["anchor"].append(row["anchor"])
        out_df["para"].append(row["para"])
        
        if row["neg_1"] != "":
            neg_count = 2
            out_df["neg1"].append(row["neg_1"])
        else:
            neg_count=1

        s = re.sub(r'[^a-zA-Z0-9\s]', ' ', row["anchor"])

        res = batch_ret(s)
        if res.shape[0] < 10:
            print(res)
        for res_num ,r in res.iterrows():
            idx_num, sent = r["docno"].split("_", 1)[0],  r["docno"].split("_", 1)[1]
            if int(idx_num) != idx_row:
                out_df[f"neg{neg_count}"].append(anchors_with_para.loc[int(idx_num)][sent])
                neg_count += 1
            if neg_count == 7:
                break
        if neg_count < 7:
            for n in range(neg_count, 7):
                out_df[f"neg{n}"].append("")

    out_df = pd.DataFrame(out_df)
    out_df.to_csv(f"out_{split}.csv")

            docno                                               text
0        0_anchor  In Paris , in October 1560 , he secretly met t...
1          0_para  In October 1560, he secretly met with English ...
2         0_neg_1  In October 1560 , he secretly met with the Eng...
3        1_anchor  The NBA season of 1975 -- 76 was the 30th seas...
4          1_para  The 1975 -- 76 season of the National Basketba...
...           ...                                                ...
5833    1998_para  Oconto is a village in Nebraska's Custer Count...
5834   1998_neg_1  Custer County , Nebraska , United States of Am...
5835  1999_anchor  In recognition of its history , its administra...
5836    1999_para  Cambridge was granted its city charter in 1951...
5837   1999_neg_1                                                   

[5838 rows x 2 columns]
16:07:56.080 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1_neg_1) - further warnings are suppresse

In [10]:
import re

/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)


  qid  docid        docno  rank      score  \
0   1   2589   883_anchor     0  15.489986   
1   1   2591    883_neg_1     1  15.489986   
2   1   2590     883_para     2  13.074037   
3   1   5382  1844_anchor     3   2.687711   
4   1   5384   1844_neg_1     4   2.687711   
5   1   5383    1844_para     5   2.636588   

                                               query  
0  Pidoux appeared as cellist Pablo Larra n in   ...  
1  Pidoux appeared as cellist Pablo Larra n in   ...  
2  Pidoux appeared as cellist Pablo Larra n in   ...  
3  Pidoux appeared as cellist Pablo Larra n in   ...  
4  Pidoux appeared as cellist Pablo Larra n in   ...  
5  Pidoux appeared as cellist Pablo Larra n in   ...  


/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)


  qid  docid        docno  rank      score  \
0   1   3975  1358_anchor     0  14.822788   
1   1   3976    1358_para     1  14.822788   
2   1   5502  1887_anchor     2   5.027579   
3   1   1167   395_anchor     3   4.528441   
4   1   1168     395_para     4   4.528441   
5   1   3429  1172_anchor     5   4.329036   
6   1   3430    1172_para     6   4.321682   
7   1   3431   1172_neg_1     7   4.321682   

                                               query  
0  Yarde sometimes improvised   often while liste...  
1  Yarde sometimes improvised   often while liste...  
2  Yarde sometimes improvised   often while liste...  
3  Yarde sometimes improvised   often while liste...  
4  Yarde sometimes improvised   often while liste...  
5  Yarde sometimes improvised   often while liste...  
6  Yarde sometimes improvised   often while liste...  
7  Yarde sometimes improvised   often while liste...  


/usr/local/lib/python3.7/dist-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)


In [32]:
for key,value in out_df.items():
    print(len(value))

1946
1946
1946
1946
1946
1946
1945
1945
